In [1]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
import rasterstats

In [1]:
#importing demmand data
gas_lau1_con_all=pd.read_csv(snakemake.input["gas_lau1"],
encoding = "ISO-8859-1", header=0, index_col=[1,3])

gas_lau1_con_all.iloc[:, 4:16]= gas_lau1_con_all.iloc[:,4:16].apply(pd.to_numeric, errors='coerce')

#import grided population data
with rasterio.open(snakemake.input["pop_uk"]) as pop_grid:
        array = pop_grid.read(1)
        crs = pop_grid.crs 
        affine = pop_grid.transform
        
#importing Calliope model zones 
calliope_zones=gpd.read_file(snakemake.input["calliope_zones"], indexcol=0)

#importing shapefile of local authorities
uk_lau1_units=gpd.read_file(snakemake.input["lau1_units"])
uk_lau1_units = uk_lau1_units.set_index("LAD19NM", drop=False)

#only select data from specified year
gas_lau1_con= gas_lau1_con_all.query('Year == @snakemake.params.year').reset_index()

#join on shapefile
gas_lau1_con.set_index('la.name', inplace=True)
lau_gas_shape = uk_lau1_units.join(gas_lau1_con)

NameError: name 'pd' is not defined

In [3]:
#overlay
lau_zones = gpd.overlay(calliope_zones.to_crs(crs), lau_gas_shape.to_crs(crs), how='intersection', keep_geom_type=False)

#calculate population per snippet
pop = rasterstats.zonal_stats(lau_zones.to_crs(crs), array, affine=affine, stats='sum', nodata=0)
lau_zones['population'] = [i['sum'] for i in pop]

# weighted average according to population
lau_zones = lau_zones.set_index(['LAD19NM', 'LAD19CD'], drop=False)
weights = lau_zones.population.div(lau_zones.sum(level='LAD19CD').population)
lau_zones['domestic_weighted']=lau_zones['domestic.sales.gwh']*weights
lau_zones['non-domestic_weighted']=lau_zones['non.domestic.sales.gwh']*weights

In [2]:
#sum up per zone and export
gas_per_zone=lau_zones.loc[:,['domestic_weighted','non-domestic_weighted']].groupby(level='LAD19NM').sum()
gas_per_zone.to_csv(snakemake.output[0])

NameError: name 'lau_zones' is not defined